#### Season 9 quick exploration

In [1]:
import pandas as pd
import altair as alt

In [2]:
# we will use this function later in the notebook
def returnUniqueCounts(dframe):
    return pd.DataFrame.from_records([(col, dframe[col].nunique()) for col in dframe.columns],
                          columns=['Column_Name', 'Num_Unique']).sort_values(by=['Num_Unique'])

Read in the CSV file that was created using an R procedure.  TERRA-REF publishes how to query through an R interface, so I used this process to generate the CSV.  Then the CSV is read in below and processing continues using Python and PANDAS.  The CSV file is an R "long type" where each measurement is in its own row, under the heading "trait" and its value is in the corresponding "mean" column. 

In [4]:
s_df = pd.read_csv('/Users/curtislisle/Dropbox/ipython-notebooks/D3M/TERRA/terraref_r/season9date.csv')

In [5]:
s_df.head()['sitename']

0     MAC Field Scanner Season 9 Range 40 Column 3
1     MAC Field Scanner Season 9 Range 40 Column 8
2     MAC Field Scanner Season 9 Range 40 Column 9
3    MAC Field Scanner Season 9 Range 40 Column 10
4    MAC Field Scanner Season 9 Range 42 Column 11
Name: sitename, dtype: object

there seems to be no East or West measurements.  Couldn't find an E or W in the sitename

In [7]:
eastern = s_df.loc[s_df.sitename.str.contains('W')]
print(eastern.shape)
eastern.head()['sitename']


(0, 40)


Series([], Name: sitename, dtype: object)

In [8]:
s_df[['trans_date','sitename','trait','mean']].head(8)

,trans_date,sitename,trait,mean
0,2019-07-25,MAC Field Scanner Season 9 Range 40 Column 3,aboveground_dry_biomass,11020.00
1,2019-07-25,MAC Field Scanner Season 9 Range 40 Column 8,aboveground_dry_biomass,17270.00
2,2019-07-26,MAC Field Scanner Season 9 Range 40 Column 9,aboveground_dry_biomass,18200.00
3,2019-07-26,MAC Field Scanner Season 9 Range 40 Column 10,aboveground_dry_biomass,13750.00
4,2019-07-26,MAC Field Scanner Season 9 Range 42 Column 11,aboveground_biomass_moisture,77.37
5,2019-07-25,MAC Field Scanner Season 9 Range 42 Column 15,aboveground_biomass_moisture,79.48
6,2019-07-25,MAC Field Scanner Season 9 Range 43 Column 2,aboveground_biomass_moisture,76.52
7,2019-07-25,MAC Field Scanner Season 9 Range 43 Column 3,aboveground_biomass_moisture,75.43


In [10]:
selected = ['id','cultivar','cultivar_id','date','trans_date','sitename','trait','mean','units']
s9sel = s_df[selected]

 If all the measurements were equally distributed, doing a long to wide rollup mechanically using pandas' pivot would work.  However, some measurements started later and ended earlier.  Some measurements are daily, some are hourly (just in August), so we really need to split up this dataset into major subsets:  daily and hourly, then try to pivot these datasets.  Or worse, have to hand convert the entries.  I elected to just write a custom algorithm to gather all the measurements together, indexed by date. 

Write a routine that pivots/rolls up the data by hand, by creating a dictionary with trans_date as its index.  Then we can add measurements one at a time...  This takes a few minutes to run on a circa-2020 CPU.  There is an update line printed every 5k entries processed.  As of when this was released, there were xxxk entries total. 

In [11]:
s9hand = {}
count = 0
for i in range(len(s9sel)):
    #if count > 40:
    #    break
    #print(i,s9sel['trans_date'][i])
    
    # if we have never seen this date before, start a new dictionary at this date
    if s9sel['trans_date'][i] not in s9hand.keys():
        s9hand[s9sel['trans_date'][i]] = {}

    # if we have not seen this cultivar before on this date, then add a dictionary for this cultivar.  Is there is a chance we 
    # might lose records here? 
    if s9sel['cultivar_id'][i] not in s9hand[s9sel['trans_date'][i]].keys():
        s9hand[s9sel['trans_date'][i]][s9sel['cultivar_id'][i]] = {}
        
    # add this feature to the dictionary for the correct cultivar on this date.  We add a dictionary entry named 
    # from the contents in the 'trait' attribute and pull the value from the 'mean' attribute.  This is the heart
    # of the long to wide format conversion.
    s9hand[s9sel['trans_date'][i]][s9sel['cultivar_id'][i]][s9sel['trait'][i]] = s9sel['mean'][i]
    
    # add the cultivar and the location (split out from the sitename text).  This will be added multiple times,
    # so represents redundant processing, but it works to place the measurements in cultivar and location
    s9hand[s9sel['trans_date'][i]][s9sel['cultivar_id'][i]]['cultivar_id'] = s9sel['cultivar_id'][i]
    s9hand[s9sel['trans_date'][i]][s9sel['cultivar_id'][i]]['cultivar'] = s9sel['cultivar'][i]
    s9hand[s9sel['trans_date'][i]][s9sel['cultivar_id'][i]]['season'] = int(s9sel['sitename'][i].split(' ')[4])
    s9hand[s9sel['trans_date'][i]][s9sel['cultivar_id'][i]]['range'] = int(s9sel['sitename'][i].split(' ')[6])
    s9hand[s9sel['trans_date'][i]][s9sel['cultivar_id'][i]]['column'] = int(s9sel['sitename'][i].split(' ')[8])
    count += 1
    if (count % 5000) == 0:
        print('in process:',count, 'records ingested so far')
print('entered ',count, 'measurements')


entered  2119 measurements


In [13]:
# there seem to be only three different measurement days
s9hand.keys()

dict_keys(['2019-07-25', '2019-07-26', '2019-07-17'])

In [18]:
# what cultivars were measured on the days?
print(len(s9hand['2019-07-17'].keys()))
print(len(s9hand['2019-07-25'].keys()))
print(len(s9hand['2019-07-26'].keys()))

12
161
146


In [21]:
s9hand['2019-07-17'].keys()


dict_keys([6000001736, 6000001772, 6000001802, 6000001869, 6000001847, 6000001747, 6000001818, 6000001775, 6000001792, 6000001739, 6000001732, 6000001771])

In [23]:
print(s9hand['2019-07-17'][6000001736])
print(s9hand['2019-07-25'][6000001736])

{'canopy_height': 156.0, 'cultivar_id': 6000001736, 'cultivar': '27.5_208_(GRASSL*RIO)', 'season': 9, 'range': 20, 'column': 8}
{'aboveground_biomass_moisture': 75.72, 'cultivar_id': 6000001736, 'cultivar': '27.5_208_(GRASSL*RIO)', 'season': 9, 'range': 48, 'column': 2, 'aboveground_fresh_biomass': 20240.0, 'aboveground_dry_biomass': 5430.0}


Sometimes the cultivar numeric index is used as the cultivar_id, sometimes it has the character name (e.g. 'cultivar_id': 6000000962,
  'cultivar': 'PI570254').  Hand measurements during August seem to have the cultivar_id entered as the cultivar, so build a cross reference by traversing the dictionary

In [24]:
cultivar_xref = {}
for key in s9hand.keys():
    # look in all the records on this date and record any names we find
    for thiscultivar in s9hand[key]:
        # now we are looking at the dictionary of measurements for a single cultivar. If the _id and cultivar name don't match, we can
        # learn from this record, so make an entry in the cross reference dictionary
        if s9hand[key][thiscultivar]['cultivar'] != s9hand[key][thiscultivar]['cultivar_id']:
            cultivar_xref[s9hand[key][thiscultivar]['cultivar']] = s9hand[key][thiscultivar]['cultivar_id']
            cultivar_xref[s9hand[key][thiscultivar]['cultivar_id']] = s9hand[key][thiscultivar]['cultivar']
    

In [25]:
# lets see what our bi-directional index looks like.  There are a few odd names, but this doesn't show unless 
# we print the whole dictionary.  Lets just sample the dictionary for reasonability
count = 0
for key in cultivar_xref:
    print(key, cultivar_xref[key])
    count += 1
    if count>15:
        break

27.5_102_(GRASSL*RIO) 6000001878
6000001878 27.5_102_(GRASSL*RIO)
27.5_211_(GRASSL*RIO) 6000001875
6000001875 27.5_211_(GRASSL*RIO)
27.5_069_(GRASSL*RIO) 6000001732
6000001732 27.5_069_(GRASSL*RIO)
27.5_162_(GRASSL*RIO) 6000001783
6000001783 27.5_162_(GRASSL*RIO)
27.5_035_(GRASSL*RIO) 6000001712
6000001712 27.5_035_(GRASSL*RIO)
27.5_308_(GRASSL*RIO) 6000001856
6000001856 27.5_308_(GRASSL*RIO)
27.5_348_(GRASSL*RIO) 6000001744
6000001744 27.5_348_(GRASSL*RIO)
27.5_208_(GRASSL*RIO) 6000001736
6000001736 27.5_208_(GRASSL*RIO)


In [33]:
widths = []
for key in s9hand.keys():
    # how many measurements are on this datetime. accumulate in a histogram dictionary
    cultivars = s9hand[key]
    # check just the first entry.  We are assuming they are all the same width.  This is probably naive, but it will get us something
    for cultivar in cultivars:
        measurement_width = len(s9hand[key][cultivar].keys())
        widths.append({'width': measurement_width})
        break
print('we found the tuple width of',len(widths), 'different measurements')
width_df = pd.DataFrame.from_records(widths)

we found the tuple width of 3 different measurements


In [35]:
alt.Chart(width_df,title="Histogram of tuple widths").mark_bar().encode(
    alt.X("width:Q", bin=True),
    y='count()',
)

alt.Chart(...)

Looking at the above histogram, there are a lot of short tuples (these are probably the height, leaf information automaticaly recorded) but 1/3 of the records are long, containing many fields).  We have seen some of those records above already.  Try to look at the widths and when they are captured.  This might not be a correct rendering:

In [36]:
alt.Chart(width_df.reset_index(),title="show the size of the tuples in date order").mark_line().encode(
    alt.Y("width:Q"),
    alt.X('index:T')
)

alt.Chart(...)

In [30]:
firstList = []
dateList = []
for key in s9hand.keys():
    print('date: ',key)
    cultivar_keys = s9hand[key].keys()
    print('cultivar-keys: ',cultivar_keys)
    for k in cultivar_keys:
        record = s9hand[key][k]
        record['cultivar_id'] = k
        # look up the textual name of the cultivar so we can match against the tree
        record['cultivar'] = cultivar_xref[k]
        record['date'] = key    
        firstList.append(record)
        dateList.append(key)
        #break
print(len(firstList))

date:  2019-07-25
cultivar-keys:  dict_keys([6000001878, 6000001875, 6000001732, 6000001783, 6000001712, 6000001856, 6000001744, 6000001736, 6000001835, 6000001843, 6000001759, 6000001730, 6000001854, 6000001716, 6000001740, 6000001752, 6000001767, 6000001770, 6000001773, 6000001824, 6000001760, 6000001842, 6000001866, 6000001757, 6000001871, 6000001849, 6000001851, 6000001806, 6000001718, 6000001803, 6000001762, 6000001814, 6000001788, 6000001836, 6000001787, 6000001808, 6000001798, 6000001054, 6000001828, 6000001750, 6000001810, 6000001733, 6000001719, 6000001861, 6000001784, 6000001831, 6000001769, 6000001779, 6000001743, 6000001772, 6000001745, 6000001714, 6000001724, 6000001728, 6000001838, 6000001853, 6000001797, 6000001809, 6000001738, 6000001867, 6000001827, 6000001801, 6000001711, 6000001739, 6000001837, 6000001846, 6000001839, 6000001756, 6000001816, 6000001749, 6000001747, 6000001847, 6000001844, 6000001800, 6000001864, 6000001723, 6000001825, 6000001873, 6000001811, 6000001

In [32]:
firstList[300]

{'aboveground_dry_biomass': 15650.0,
 'cultivar_id': 6000001847,
 'cultivar': '27.5_316_(GRASSL*RIO)',
 'season': 9,
 'range': 12,
 'column': 12,
 'aboveground_biomass_moisture': 81.23,
 'aboveground_fresh_biomass': 83380.0,
 'date': '2019-07-26'}

In [37]:
import pandas as pd
full_df = pd.DataFrame(firstList,index=dateList)
full_df.head()

,aboveground_biomass_moisture,aboveground_dry_biomass,aboveground_fresh_biomass,canopy_height,column,cultivar,cultivar_id,date,range,season
2019-07-25,70.80,9255.0,33770.0,NaN,15,27.5_102_(GRASSL*RIO),6000001878,2019-07-25,14,9
2019-07-25,77.43,8619.0,38180.0,NaN,5,27.5_211_(GRASSL*RIO),6000001875,2019-07-25,23,9
2019-07-25,81.75,7997.0,73930.0,NaN,15,27.5_069_(GRASSL*RIO),6000001732,2019-07-25,42,9
2019-07-25,81.96,7003.0,64610.0,NaN,2,27.5_162_(GRASSL*RIO),6000001783,2019-07-25,43,9
2019-07-25,78.29,7885.0,32090.0,NaN,3,27.5_035_(GRASSL*RIO),6000001712,2019-07-25,43,9


In [38]:
returnUniqueCounts(full_df)

,Column_Name,Num_Unique
9,season,1
7,date,3
3,canopy_height,12
4,column,14
8,range,50
5,cultivar,169
6,cultivar_id,169
0,aboveground_biomass_moisture,267
1,aboveground_dry_biomass,282
2,aboveground_fresh_biomass,303


In [41]:
full_df.to_csv('s9_height_biomass.csv',index=False)